In [ ]:
!wget -O "openTECR recuration - actual data.csv" "https://docs.google.com/spreadsheets/d/1jLIxEXVzE2SAzIB0UxBfcFoHrzjzf9euB6ART2VDE8c/export?format=csv&gid=2123069643"
!wget -O "openTECR recuration - table codes.csv" "https://docs.google.com/spreadsheets/d/1jLIxEXVzE2SAzIB0UxBfcFoHrzjzf9euB6ART2VDE8c/export?format=csv&gid=831893235"
!wget -O "openTECR recuration - table metadata.csv" "https://docs.google.com/spreadsheets/d/1jLIxEXVzE2SAzIB0UxBfcFoHrzjzf9euB6ART2VDE8c/export?format=csv&gid=1475422539"
!wget -O "openTECR recuration.ods" "https://docs.google.com/spreadsheets/d/1jLIxEXVzE2SAzIB0UxBfcFoHrzjzf9euB6ART2VDE8c/export?format=ods"
!wget -O "TECRDB.csv" "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv"
##Downloading openTECR and Noor data

--2025-01-17 18:59:21--  https://docs.google.com/spreadsheets/d/1jLIxEXVzE2SAzIB0UxBfcFoHrzjzf9euB6ART2VDE8c/export?format=csv&gid=2123069643
Resolving docs.google.com (docs.google.com)... 74.125.128.100, 74.125.128.113, 74.125.128.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.128.100|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-00-8k-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/73nbh4anccgrg7oplu9esnjpjo/1737140360000/115120384215235060766/*/1jLIxEXVzE2SAzIB0UxBfcFoHrzjzf9euB6ART2VDE8c?format=csv&gid=2123069643 [following]
--2025-01-17 18:59:21--  https://doc-00-8k-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/73nbh4anccgrg7oplu9esnjpjo/1737140360000/115120384215235060766/*/1jLIxEXVzE2SAzIB0UxBfcFoHrzjzf9euB6ART2VDE8c?format=csv&gid=2123069643
Resolving doc-00-8k-sheets.googleusercontent.com (doc-00-8k-sheets.googleusercontent.com)... 173.194.69.132, 2a00:1450:4013:c04::84
C

In [ ]:
!pip install odfpy
## Installing library for processing .ods file

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160672 sha256=21887d614af743c8b1238dc15d7e101e0de368e5d08284e9c144872a5f072e2a
  Stored in directory: /root/.cache/pip/wheels/d6/1d/c8/8c29be1d73ca42d15977c75193d9f39a98499413c2838ac54c
Successfully built odfpy


In [ ]:
import pandas

READ_CSV = True


## data upload
if READ_CSV:
    df = pandas.read_csv("openTECR recuration - actual data.csv")
else:
    df = pandas.read_excel("openTECR recuration.ods", sheet_name="actual data")
df = df.replace({"col l/r": {"l":1,"r":2}}) ##replaced l and r in col l/r with 1 and 2

## containing NaNs
print("containing NaNs:")

#script that counts rows containing NaN values
subset = ["part", "page", "col l/r", "table from top", "entry nr"]
counter = df[subset].isna().any(axis=1).sum()
print(f"A total of {counter} rows contained NaNs.")

containing NaNs:
A total of 10 rows contained NaNs.


In [ ]:
## QC

## Checking for duplicates
## Replaced reference with reference_code as it is the real column name
## Not actually needed anymore due ot duplicable table column

if True:
    ## duplicates and errors
    test_df = df
    MANUALLY_EXCLUDED_DUPLICATES = [
        "54STA",
        "71TAN/JOH",
        "91HOR/UEH",
        "76SCH/KRI",
        "99TEW/SCH"
    ]
    ## shouldn't be necessary after introduction of duplicate_table column; so we use:
    MANUALLY_EXCLUDED_DUPLICATES = []
    ##
    test_df = test_df[~test_df.reference_code.isin(MANUALLY_EXCLUDED_DUPLICATES)]
    assert sum(test_df[((test_df["entry nr"]=="duplicate") | (test_df["entry nr"]=="error"))].id.isna())==0, ("Duplicate or error found for an empty-ID row", test_df[(((test_df["entry nr"]=="duplicate") | (test_df["entry nr"]=="error"))) & test_df.id.isna()])
    ## Ensures that rows containing duplicate or error in entry nr column also has an appropriate ID.

#print("I am removing the following duplicates and errors:")
df = df[~((df["entry nr"]=="duplicate") | (df["entry nr"]=="error"))]
## Removes such rows

In [ ]:
#QC
if True:
    ##check completeness of position annotation
    na_counter = df[["part","page","col l/r","table from top", "entry nr"]].isna().sum(axis="columns") ##Counts NaN values in position annotation
    #print(na_counter)
    assert len(df[~na_counter.isin([0,5])])==0, print(df[~na_counter.isin([0,5])][["id","reference","part","page","col l/r","table from top", "entry nr"]].to_string())
    #Prints problematic rows if any

    ##Checks for spaces in /reference_code/ column.
    assert len(df[df.reference_code.str.contains(" ").fillna(False)])==0, print(df[df.reference_code.str.contains(" ").fillna(False)].to_string())
    ## And output problematic rows.

## drop NaNs -- these entries just haven't been worked on and can't be checked
df = df.dropna(subset=["part","page","col l/r","table from top", "entry nr"])

## convert values in columns part, page, col l/r, table from top and entry nr to integers
df[["part","page","col l/r","table from top", "entry nr"]] = df[["part","page","col l/r","table from top", "entry nr"]].astype(int)

<ipython-input-57-3d14777ca5e3>:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  assert len(df[df.reference_code.str.contains(" ").fillna(False)])==0, print(df[df.reference_code.str.contains(" ").fillna(False)].to_string())


In [ ]:
## quality check new data

if True:
    ## id is unique
    assert len(df.dropna(subset=["id"]).id.unique()) == len(df.dropna(subset=["id"])), df.dropna(subset=["id"])[df.dropna(subset=["id"]).id.duplicated()].to_string()
    ## Ensures that every value in column ID is unique, if not prints out problematic tows.

    ## tables intact in themselves

    ##Groups rows in dataframe by part, page, col l/r, and table from top values.

    for which, g in df.groupby(["part","page","col l/r","table from top"]): # g is subset corresponding to specific part of the dataframe, as grouped by previously
        #print((which,g))
        #if which == (3, 1091, 1, 1):
        #    continue

        assert len(g.reference_code.unique())==1, (which, print(g.to_string())) ## Checks that all values in reference_code have the same value--
        assert len(g.EC.unique()) == 1, (which, print(g.to_string())) ## Check that all EC values are uniform
        assert len(g.reaction.unique())==1, (which, print(g.to_string())) ## - fixed "description" for correct column name
        assert sorted(g["entry nr"].values)==list(range(1,g["entry nr"].max()+1)), (which, print(g.to_string())) ## Check that all values in entry nr are ordered from lowest to highest

    ## table counts consistently continuous
    for which, g in df.groupby(["part", "page", "col l/r"]):
        ## the following part/page/column combinations
        #
        # contain a table which was mentioned before;
        ## the table is thus marked full duplicate and was removed beforehand; so we manually
        ## exclude those from this automated check
        MANUALLY_EXCLUDED_COLUMNS = [
            (2, 558, 2),
            (2, 560, 2),
            (2, 566, 2),
            (2, 584, 1),
            (2, 590, 2),
            (3, 1041, 1),
            (3, 1076, 2),
            (7, 1360, 2),
            (7, 1369, 2),
            (7, 1373, 1),
        ]
        ## shouldn't be necessary after introduction of duplicate_table column; so we use:
        MANUALLY_EXCLUDED_COLUMNS = []
        ## because there is a peculiarity about tables not-existent-in-the-pdf, but found in randr, we have to use:
        MANUALLY_EXCLUDED_COLUMNS = [
            (6, 948, 2),
            (6, 949, 1),
        ]
        if which in MANUALLY_EXCLUDED_COLUMNS:
            continue
        assert sorted(g["table from top"].unique()) == list(range(1, g["table from top"].max() + 1)), (which, print(g.to_string())) # ensures that values in table from top are from lowest to highest

    ## column values either 1 or 2
    for which, g in df.groupby(["part", "page"]):
        assert all([i in [1,2] for i in g["col l/r"].values]), (which, print(g.to_string())) # ensures that all values in col l/r are either 1 or 2

    ## page numbers consistently continuous
    for which, g in df.groupby("part"):
        MANUALLY_EXCLUDED_PARTS = [
            #2,
            #3,
            #7
        ]
        if which in MANUALLY_EXCLUDED_PARTS: #ensures that page numbers are continious by parts
            continue

        print(f"----- This is about part {which} -----")
        should_be = list(range(g["page"].min(), g["page"].max() + 1))
        for page in should_be:
            if page not in g["page"].unique():
                print(f"Missing page: {page}") #Finds missing pages and prints them out
        #assert sorted(g["page"].unique()) == list(range(g["page"].min(), g["page"].max() + 1))

## extract added values
# print(f"You will need to care for these {df.id.isna().sum()} recently added rows:")
# print(df[df.id.isna()])

print("The online spreadsheet data looks consistent.")

----- This is about part 1 -----
----- This is about part 2 -----
----- This is about part 3 -----
Missing page: 1088
Missing page: 1089
Missing page: 1090
----- This is about part 4 -----
----- This is about part 5 -----
----- This is about part 6 -----
----- This is about part 7 -----
The online spreadsheet data looks consistent.


In [ ]:
## consistency check between online spreadsheet and original Noor data
##Reading TECRDB and openTECR files
noor = pandas.read_csv("TECRDB.csv")
noor = noor.rename(columns={"reaction": "keggID"})
noor = noor.rename(columns={"reference": "reference_code",'description':'reaction'})
## Updating column names in Noor set to match the new columns in openTECR.
if READ_CSV:
    online = pandas.read_csv("openTECR recuration - actual data.csv")
else:
    online = pandas.read_excel("openTECR recuration.ods", sheet_name="actual data")


online = online.replace({"col l/r": {"l":1,"r":2}}) #replacing l/r values with 1 and 2
## check that all ids are still there
assert set(noor.id) - set(online.id) == set(), f"The following IDs were deleted online: {set(noor.id)-set(online.id)}"
## Checks that all ID are present in both openTECR and Noor

## non-curated values should not have been changed by anyone!
leftjoined = pandas.merge(noor, online.dropna(subset="id"), on="id", how="left", validate="1:1")
## Merging noor set and openTECR together

In [ ]:
## This code snippet tries to check if K_prime values are equal in Noor and openTECR dataset
## While accounting for rounding up in openTECR
## If values are not equal despite being rounded up, it will output both k_prime values from noor and openTECR
## kPn is a prime_K column from Noor set roughly rounded up by the rules outlined in openTECR



import numpy as np
kPn = []
for i in range(len(leftjoined.K_prime_x)):
  if leftjoined.K_prime_x[i] != leftjoined.K_prime_y[i]:
    if np.isnan(leftjoined.K_prime_x[i]) == False:
      #print(leftjoined.K_prime_x[i])
      #print(leftjoined.K_prime_y[i])
      ##Trying to match weird rounding up format of openTECR Kprime values
      if len(str(leftjoined.K_prime_x[i]).split('.')[1])>4:
        rounded = round(leftjoined.K_prime_x[i],len(str(leftjoined.K_prime_x[i]).split('.')[1])-1)
        #print(rounded)
      elif len(str(leftjoined.K_prime_x[i]).split('.')[1])==2:
        rounded = round(leftjoined.K_prime_x[i],1)
        #print(rounded)
      elif len(str(leftjoined.K_prime_x[i]).split('.')[1])==3:
        rounded = round(leftjoined.K_prime_x[i], 2)
        #print(rounded)
      elif len(str(leftjoined.K_prime_x[i]).split('.')[1])==1:
        rounded = round(leftjoined.K_prime_x[i], 0)
        #print(rounded)
      #print(round(leftjoined.K_prime_x[i], 2))
      if rounded+0.1 == leftjoined.K_prime_y[i]:
        kPn.append(rounded+0.1)
      else:
        kPn.append(rounded)
    else:
      kPn.append(leftjoined.K_prime_x[i])
  else:
    kPn.append(leftjoined.K_prime_x[i])
for i in range(len(leftjoined.K_prime_y)):
  if kPn[i] != leftjoined.K_prime_y[i]:
    if np.isnan(leftjoined.K_prime_y[i]) == False:
      print(f'Noor set K_prime value: {kPn[i]}')
      print(f'openTECR set K_prime value: {leftjoined.K_prime_y[i]}')
## Some cases couldn't be handled since they don't follow rounding logic

In [ ]:
## Columns that should be the same
SHOULD_BE_THE_SAME = [
    "reference_code",
    "EC",
    "reaction",
    "K",
    "temperature",
    "ionic_strength",
    "p_h",
    "p_mg",
    #"K_prime", # - checked manually, weird rounding up in online data? Handled by code above
]
for s in SHOULD_BE_THE_SAME:
    entries_where_both_are_nans = leftjoined[ leftjoined[f"{s}_x"].isna() & leftjoined[f"{s}_y"].isna() ]
    if len(entries_where_both_are_nans) == 0:
        assert (leftjoined[f"{s}_x"] == leftjoined[f"{s}_y"]).all(), (s, print(leftjoined[~(leftjoined[f"{s}_x"] == leftjoined[f"{s}_y"])][["id",f"{s}_x",f"{s}_y"]].to_string()))
    else:
        tmp = leftjoined[ ~ (leftjoined[f"{s}_x"].isna() & leftjoined[f"{s}_y"].isna()) ]
        assert (tmp[f"{s}_x"] == tmp[f"{s}_y"]).all(), (s, print(tmp[~(tmp[f"{s}_x"] == tmp[f"{s}_y"])][["id",f"{s}_x",f"{s}_y"]].to_string()))

## The code above checks that all values in those columns between both Noor (_x) and openTECR (_y) are either NaN or equal. If not, raises assertion error and prints out incorrect rows

## did someone add a new row without id, where they should have corrected a row with id?
merged = pandas.merge(online, noor, on=[
    "reference_code",
    "temperature",
    "ionic_strength",
    "p_h",
    "p_mg",
    "K_prime",
])
merged = merged[merged["id_x"] != merged["id_y"]]
potential_errors = merged[merged["id_x"].isna() | merged["id_y"].isna()]

## Checks whether or not noor and online set have matching ID values

## the following have been manually checked to be able to be excluded from the comparison below:
MANUALLY_EXCLUDED = [
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry4356",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry1714",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry1715",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry1716",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry1718",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry1717",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry2140",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry2149",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry3167",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry3184",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry707",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry4246",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry392",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry1269",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry4283",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry4284",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry3888",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry3896",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry3897",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry1915",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry1603",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry1605",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry236",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry386",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry2718",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry3560",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry3561",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry4006",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry1271",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry1586",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry1589",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry1590",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry1591",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry1588",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry1587",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry2370",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry2371",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry1601",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry3637",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry2725",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry705",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry702",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry704",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry4040",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry4373",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry1829",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry3031",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry3032",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry2339",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry1916",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry3026",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry4086",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry2848",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry4316",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry772",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry2851",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry398",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry82",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry806",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry805",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry3654",
    "https://w3id.org/related-to/doi.org/10.5281/zenodo.3978439/files/TECRDB.csv#entry2277",
]
potential_errors = potential_errors[ ~potential_errors.id_y.isin(MANUALLY_EXCLUDED) ]
if len(potential_errors) > 0:
    print("The following entries might have been added as a new row without id, where they should have corrected a row with id:")
    print(potential_errors.to_string())

print("The online spreadsheet data and its original data source are still in sync.")

The online spreadsheet data and its original data source are still in sync.


In [ ]:

## annotate online spreadsheet with table_codes
#
#
## read second df
## -- get it manually from https://zenodo.org/record/5495826 !
noor = pandas.read_csv("TECRDB.csv")
noor.columns
#Index(['id', 'url', 'reference', 'method', 'eval', 'EC', 'enzyme_name',
#       'reaction', 'description', 'K', 'K_prime', 'temperature',
#       'ionic_strength', 'p_h', 'p_mg'],

noor = noor.drop(['K', 'K_prime', 'temperature',
       'ionic_strength', 'p_h', 'p_mg'],axis="columns")

## Fixed to matching name in openTECR
noor = noor.rename(columns={"reaction": "keggID"})
noor = noor.rename(columns={"reference": "reference_code",'description':'reaction'})

## extract table codes
noor["table_code"] = noor.url.str.split("&T1=").str[-1]

## quality check
if True:
    ## tables intact in themselves
    for which, g in noor.groupby("table_code"):
        #print((which,g))
        assert len(g.reference_code.unique())==1, (which, print(g.to_string()))
        assert len(g.method.unique())==1, (which, print(g.to_string()))
        assert len(g["eval"].unique()) == 1, (which, print(g.to_string()))
        assert len(g.EC.unique()) == 1, (which, print(g.to_string()))
        assert len(g.enzyme_name.unique())==1, (which, print(g.to_string()))
        assert len(g.reaction.unique()) == 1, (which, print(g.to_string()))
        assert len(g.reaction.unique()) == 1, (which, print(g.to_string()))

## Ensures that all values in those columns are consistent

## drop now-unnecessary columns
noor = noor.drop(["EC","reference_code", "reaction"], axis="columns")
#df   = df.drop(  ["description"],    axis="columns")

## merge
tmp = pandas.merge(df, noor, how="left", on="id")

## add manually extracted table codes
if READ_CSV:
    manual_table_codes = pandas.read_csv("openTECR recuration - table codes.csv")
else:
    manual_table_codes = pandas.read_excel("openTECR recuration.ods", sheet_name="manually mapped table codes")

# QC
if True:
    assert sum(manual_table_codes.duplicated(["part", "page", "col l/r", "table from top"])) == 0, print(
        manual_table_codes[manual_table_codes.duplicated(["part", "page", "col l/r", "table from top"])])
# split into tables with table codes from Noor and those which needed to be annotated manually
manual_table_codes = manual_table_codes.drop(["reference", "description"], axis="columns")
tmp_with_table_codes = tmp[~tmp.table_code.isna()]
tmp_without_table_codes = tmp[tmp.table_code.isna()]
tmp_without_table_codes = tmp_without_table_codes.drop("table_code", axis="columns")
tmp_without_table_codes_try_to_add_manual_ones = pandas.merge(tmp_without_table_codes, manual_table_codes, how="left", on=["part","page","col l/r","table from top"])
# concat the two
new = pandas.concat([tmp_with_table_codes, tmp_without_table_codes_try_to_add_manual_ones], ignore_index=True)
## keep only one entry per table code, remove now-meaningless columns, but keep id=NaN rows
new = new[~new.duplicated(["part","page","col l/r","table from top"])]
new = new.drop(["id","url"], axis="columns")
new["comment"] = ""

## Ensures no diplicate values in manual_table_codes
## Splits tmp into portitions with and without table_codes
## Removes duplicate entries and adds column "comment" for further annotation

## export tables which need to be added to the "table codes" tab
selector = []
for i,s in new.iterrows():
    if pandas.isna(s.table_code):
        if len(manual_table_codes[(manual_table_codes.part == s.part) &
                            (manual_table_codes.page == s.page) &
                            (manual_table_codes["col l/r"] == s["col l/r"]) &
                            (manual_table_codes["table from top"] == s["table from top"])
           ]) == 0:
            selector.append(i)
export = new.loc[selector]
(export[[
    "part","page","col l/r","table from top",
    "table_code",
    "reaction",
    "reference_code",
    "curator",
    ]]
 .sort_values(["part","page","col l/r","table from top"])
 .to_csv("2024-01-06-opentecr-recuration.missing_table_codes.csv", index=False)
 )

## identifies and exports rows from the new DataFrame that are missing table_code values and do not already exist in manual_table_codes, creating a CSV file with them
## selector - rows that need to be exported


## export tables which need to have their comment extracted
selector = []
if READ_CSV:
    tables_with_comments = pandas.read_csv("openTECR recuration - table metadata.csv")
else:
    tables_with_comments = pandas.read_excel("openTECR recuration.ods", sheet_name="table comments")
## QC
if True:
    assert sum(tables_with_comments.duplicated(["part","page","col l/r","table from top"]))==0, print(tables_with_comments[tables_with_comments.duplicated(["part","page","col l/r","table from top"])])

# Identify rows in new df that are not present in the metadata
for i, s in new.iterrows():
    # Check if a matching entry exists in `tables_with_comments`
    if len(tables_with_comments[
        (tables_with_comments.part == s.part) &
        (tables_with_comments.page == s.page) &
        (tables_with_comments["col l/r"] == s["col l/r"]) &
        (tables_with_comments["table from top"] == s["table from top"])
    ]) == 0:
        # If no match is found, add the index of the row to `selector`
        selector.append(i)

tables_without_comments = new.loc[selector]
# Add placeholders for manual annotation and spellchecking
tables_without_comments["manually spellchecked"] = ""
tables_without_comments["comment"] = ""


# Export the filtered data to a CSV file

(tables_without_comments[[
    "part","page","col l/r","table from top",
    "reference_code",
    "manually spellchecked",
    "comment"
    ]]
 .sort_values(["part","page","col l/r","table from top"])
 .to_csv("2024-01-06-opentecr-recuration.missing-table-comments.csv", index=False)
 )

print("I could merge the online spreadsheet and the Noor data. I have written file regarding the tables to your disk.")